# Hierarchical Bayesian LC Models for late-time WISE data in Type IIP SN

## Motivation

Type IIP SNe are powerful probes of the final stages of massive stellar evolution. They are the most common SN (60-70% of observed SNe), come from H-rich progenitors (progenitors that have retained their H envelope), and are expected to be the end stage of the lower mass boundary of stars that undergo CCSN. 

The luminosity and shape of Type IIP SNe light curves encode many key physical quantities regarding the progenitors and the explosions themselves. Specifically, the luminosity of the radioactive tail can be used to derive the synthesized nickel mass, $M_\text{Ni}$, and thus in turn inform us about the structure of the progenitor star.

$$L \propto M_\text{Ni}e^{-t/\tau_\text{Co}}$$

The most comprehensive analyzed sample of ZTF Type IIP SNe light curves in [Das 2025b](https://arxiv.org/pdf/2506.20068) demonstrated a strong correlation between the bolometric luminosity at the plateau phase, a good indicator of the explosion energy, and the derived $M_\text{Ni}$ from the radioactive tail. However, within this comprehensive sample, there are outlier events that exhibit stronger bolometric luminosity for the plateau phase and lower derived $M_\text{Ni}$.
<p align="center">
  <img src="LvsMNi.png" />
</p>

These anomalous Type IIP SNe have dramatic cut offs in their radioactive tails. These unusually low $M_\text{Ni}$ can be potential flags for candidate partial fallback explosions, where a significant fraction of the inner ejecta fails to escape in the explosion and falls back onto the compact object remnant. To confidently flag these partial fallback candidates, one needs to rule out that the optical tail luminosity is absorbed by dust formed in the SNe and re-emitted in the mid-infrared.

<p align="center">
  <img src="ZTF22abtspsw.png" />
</p>

In my research, I am focusing on the Das sample of $\sim 100$ Type IIP SNe discovered by ZTF. The subsample of anomalous sources, with steep radioactive tail drop-offs, are prime partial fallback candidates if there faint tails cannot be explained by dust reprocessing. In order to do this I build SEDs from ZTF and WISE data and fit dust radiative transfer models. However, only a subset of the sample has contemporaneous ZTF and WISE data in the tail phase, limiting the number of SNe for which I can model the optical+IR SED directly. 

To address this, I will use a hierarchical Bayesian light-curve model for the late-time optical light curve as a way to statistically infer the $r$ band flux throughout the tail epoch for SNe that lack coincident ZTF and WISE coverage. **Using the subset of SNe with coincident ZTF and WISE data, I will fit a population-level model for the tail decline and predict the ZTF flux as a function of radioactive tail phase, allowing each SN to have its own decline behavior informed by population distributions.** This model that uses population trends will then provide informed posterior predictions for the ZTF flux at a specific tail epoch for SNe with missing ZTF data near the sparse WISE detections in the tail epoch.

Combining the inferred ZTF fluxes with the WISE photometry in the tail epoch, I can construct tail-phase optical+IR SEDs for a much larger fraction of Type IIP SNe from the total sample compared to when relying only on raw data. With this expanded sample, I can continue to model how much tail luminosity is reprocessed by dust, derive true $M_\text{Ni}$, reassess outliers after accounting for dust, and confidently flag anomalously low-$M_\text{Ni}$ objects as partial fallback candidates.

## Population-level Hierarchical Tail Model

In this project, I aim to develop and use a hierarchical Bayesian model for the optical tail light curves of Type IIP SNe. The tails can be described by a simple parametric model whose paramaraters vary across individual SN but can be informed from population distributions of those parameters.

$$\text{Theory: } \ \ L \propto M_\text{Ni}e^{(-t/\tau_\text{Co})}$$
$$\text{Model: } \ \ \log_{10}{F}=\alpha + \beta t$$
$$\text{Hierarchical Model with Parameters informed from Population Distribution: }$$
$$\log_{10}{F} \sim \mathcal{N}(\alpha_i + \beta_i t, \sigma_i)$$
$$\alpha_i \sim \mathcal{N}(\mu_\alpha, \sigma^2_\alpha)  \ ,  \ \ \beta_i \sim \mathcal{N}(\mu_\beta, \sigma^2_\beta)$$

Our theoretical tail behavior set by the decay timescale of radioactive cobalt can be well approximated linearly in log flux space. In the hierarchical formulation of this model, each SN $i$ will have a slope $\beta$ and intercept $\alpha$ constrained by the population-level distributions of these parameters (hyperparameters). In this model formulation, well-sampled SNe significantly inform the population distributions while for poorly-sampled SNe, their posterior will be pulled towards the mean of these population distributions. This phenomenon, where for sparsely-covered SNe the posterior is pulled towards the population mean, is called 'partial pooling'.

$$p(\alpha_i, \beta_i \ | \ \text{data for SN}_i, \mu_\alpha, \sigma_\alpha, \mu_\beta, \sigma_\beta) \propto p(\text{data for SN}_i \ | \ \alpha_i, \beta_i) \ \times \ p(\alpha_i, \beta_i \ | \ \mu_\alpha, \sigma_\alpha, \mu_\beta, \sigma_\beta)$$

**With a population-level hierarchical model, I hope to learn the population-level distributions for the tail parameters, namely the slope, $\beta$, and intercept, $\alpha$, use both per-SN likelihoods informed by those population distributions to find $\alpha_i$, $\beta_i$, and then generate posterior predictive distributions for the $r$-band flux throughout the tail epoch.**

## Preparing the Data

One of the most laborious processes of this project is, of course, preparing our data to feed into the hierarchical modeling procedure. For each SN in the sample I first had to identfy the radioactive tail phase, convert observed magnitudes and their uncertainties to fluxes, remove datapoints with bad SNR, and filter for $r$-band data. The $r$ band captures ZTF's peak sensitivity. Furthermore, for the modelling in `PyMC` accurate SNe indexing was required. 

For each SNe, I performed simple least-squares fits to get each SN's slope and intercept. This was done to i) provide motivated initial guesses (the means across the sample) ii) remove our anomalous SNe since their behavior is already expected to not follow the population norm.

The relevant code for this procedure can be found in the `src/tailwise/ztf.py` and `src/tailwise/hbm.py` files. In the `src/tailwise/hbm.py` file, a lot of this data cleaning and preparation happens in the `clean_data()` and `prepare_data()` methods.

## Implementing the Hierarchical Tail Model using `PyMC`

I constructed the hierarchical tail model using `PyMC`, an open source probabilistic programming library that makes it easy to build Bayesian models and fit them using MCMC. This library made it easy to set hyperpriors and the SN specific parameters. You can see this in the `build_model()` method in `TailHBModel` in `src/tailwise/hbm.py`. 

For the hierarchical structure, even though each SN has its own $\alpha_i$ and $\beta_i$, they are constrained by the population distributions:
$$\alpha_i \sim \mathcal{N}(\mu_\alpha, \sigma^2_\alpha)  \ ,  \ \ \beta_i \sim \mathcal{N}(\mu_\beta, \sigma^2_\beta)$$
which are the hyperparameters inferred from all the SNe in the sample.

For my hyperpriors, specifically the initial normal distributions of $\mu_\alpha$, $\sigma_\alpha$, $\mu_beta$, $\sigma_beta$, I take the mean of the LS fit parameters from the normal SNe sample. These are reasonable "grounding" values for which I can start fitting my model using MCMC.

After building the model, we can do inference. I fit the model in `PyMC` using their MCMC to calculate the posterior distribution of all the model parameters:
$$p(\mu_\alpha, \sigma_\alpha, \mu_\beta, \sigma_\beta, \alpha_i, \beta_i \ | \ \text{data for SN}_i)$$
This is done in the `fit_model()` method where one can tune how many MCMC chains they want, warmup steps (`tune`), posterior draws, acceptance probability requirements, and CPU cores to run across. I save my MCMC fits to `*.nc` files which you can find in `data/fits/`.

After generating the fitted posteriors, we can construct the predicted light curves for each SN! For each SN, I draw samples of $\alpha_i$ and $\beta_i$ parameters from the posterior and calculate $\log_{10}F=\alpha_i + \beta_i t$ to generate a distribution of light curves. Generally, throughout my code, I care about the 16%-50%-84% intervals for the inferred fluxes. A demo of this can be found in [`notebooks/generate_model_tail_LC.ipynb`](https://github.com/ana-lam/astroskills_final/blob/main/notebooks/generate_model_tail_LCs.ipynb).

Here is an example inferred light curve from the posterior of a model fitting.

<p align="center">
  <img src="ZTF19acbwejj_tail_model.png" />
</p>

## Assessing the Model

To assess the performance of the hierarchical Bayesian tail model and how well it can predict $r$ band fluxes late in the tail, I decided to do a simple measure of how well it would be able to predict masked/purposefully hidden observations. In [`notebooks/mode_assessment.ipynb`](https://github.com/ana-lam/astroskills_final/blob/main/notebooks/model_assessment.ipynb), I randomly masked 33% of the $r$-band detections of the SNe with the most $r$-band coverage and conducted the whole model fitting pipeline again on the filtered dataset. 

After fitting, I evaluated the predicted fluxes at the phases of those removed observations. My first pass at this yielded quite terrible results with only about $\sim 8.8 \%$ of the predicted masked detections being within the 16%-84% credible interval. However, my performance improved significantly when I included the intrinsir scatter term $\sigma_\text{int}$ that I baked into my model but did not use in my predictions from the posterior. Including $\sigma_\text{int}$ improved the performance to $\sim 58\%$.

While including $\sigma_\text{int}$ greatly improved my predictions, $\sim 58\%$ isn't exactly a high number. Of course these events are much more complicated and perhaps my model approximation, a linear decline in log-flux, is a crude approximation for something like predicting late tail optical flux. Perhaps future work would be adding some complexity to the model, introducing new parameters, incorporating the other band detections and upper limits... 


## Takeaways




This project was my first time trying Bayesian modeling in any sense and particularly with observational data which I am new to. While my original goal was to use it to solve a challenge in my first-year research project, I found myself diving into the model and whatever upgrades I could include in it. Furthermore, I became particularly interested in developing a software/data analysis project with good development practices and with flexibility top of mind. More specifically with the statistical method, I sought out and learned how to implement hierarchical modeling for this kind of problem: where I have well-sampled SNe that I wanted to leverage to address poorly-sampled SNe (the partial-pooling of it all). And though this doesn't really say much to my anomalous tail SNe, it could work as a method that further confirms these are outliers due to the partial pooling effect.

Overall, this project is a great foundation for future work. Perhaps if I tune it to the point of very precise performance, I will be able to use it to extend the SED outputs of my SNe sample, especially for the SNe that exhibit normal tails. Furthermore, as I mentioned, this might be able to further flag anomalous SNe that are of particular interest to me and my research project.

